In [1]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [3]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [5]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-03-10 21:18:07,115] A new study created in memory with name: no-name-9f7d30f6-8a89-48d1-b2a3-c57b32b7d9ce
[I 2025-03-10 21:18:07,631] Trial 0 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 121, 'max_depth': 20}. Best is trial 0 with value: 0.7802607076350093.
[I 2025-03-10 21:18:08,004] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 101, 'max_depth': 12}. Best is trial 0 with value: 0.7802607076350093.
[I 2025-03-10 21:18:08,577] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 152, 'max_depth': 17}. Best is trial 0 with value: 0.7802607076350093.
[I 2025-03-10 21:18:09,327] Trial 3 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 197, 'max_depth': 6}. Best is trial 0 with value: 0.7802607076350093.
[I 2025-03-10 21:18:10,005] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 168, 'max_depth': 12}. Best is trial 0 with value: 0.78026

In [6]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 73, 'max_depth': 17}


In [7]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


Samplers in Optuna

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [9]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-03-10 21:18:31,210] A new study created in memory with name: no-name-2f35aa0a-9021-4b41-85fb-8800a1232fd3
[I 2025-03-10 21:18:31,691] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 140, 'max_depth': 12}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-03-10 21:18:32,215] Trial 1 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 133, 'max_depth': 16}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-03-10 21:18:32,823] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 178, 'max_depth': 9}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-03-10 21:18:33,347] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 149, 'max_depth': 15}. Best is trial 1 with value: 0.7802607076350093.
[I 2025-03-10 21:18:33,947] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 185, 'max_depth': 7}. Best is trial 1 with value: 0.780260

In [10]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7839851024208566
Best hyperparameters: {'n_estimators': 73, 'max_depth': 18}


In [11]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


In [12]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [13]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-03-10 21:19:00,533] A new study created in memory with name: no-name-edac8c15-7df1-459f-823a-77e34691d32c
[I 2025-03-10 21:19:01,309] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-03-10 21:19:02,690] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-03-10 21:19:03,167] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-03-10 21:19:03,994] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-03-10 21:19:04,673] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [14]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [15]:
%pip install plotly
import plotly

Note: you may need to restart the kernel to use updated packages.


In [16]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [17]:
# 1. Optimization History
plot_optimization_history(study).show()

In [18]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [19]:
# 3. Slice Plot
plot_slice(study).show()

In [20]:
# 4. Contour Plot
plot_contour(study).show()

In [21]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()


Optimizing Multiple ML Models

In [22]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [23]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [31]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100)

[I 2025-03-10 21:27:46,493] A new study created in memory with name: no-name-87fe9ae5-4b7a-49ec-89c4-1eade614798b
[I 2025-03-10 21:27:46,913] Trial 0 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 129, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-03-10 21:27:46,938] Trial 1 finished with value: 0.739292364990689 and parameters: {'classifier': 'SVM', 'C': 3.8266757706921313, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-03-10 21:27:47,619] Trial 2 finished with value: 0.7467411545623838 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 189, 'learning_rate': 0.24240490412030322, 'max_depth': 3, 'min_samples_split': 4, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-03-10 21:27:48,448] Trial 3 finished with value: 0.7616387337057727 and parameters:

In [25]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11261793481399071, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [26]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 79
RandomForest        11
GradientBoosting    10
Name: count, dtype: int64

In [27]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.746555
RandomForest        0.763840
SVM                 0.775711
Name: value, dtype: float64

In [28]:
# 1. Optimization History
plot_optimization_history(study).show()


In [29]:
# 3. Slice Plot
plot_slice(study).show()

In [30]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()